# Import Required Libraries
Import PyTorch, torchvision, and other necessary libraries.

In [89]:
# Import Required Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms, models
from torch.utils.data import random_split
import os
import time
import copy


# Set Data Directory
Define the directory path for the dataset.

In [90]:
# Set data directory
data_dir = 'D:\\neuralnetwork\\Ex2\\flower_dataset'

# Data Augmentation and Normalization
Apply data augmentation techniques and normalize the dataset using torchvision.transforms.

In [91]:
# Data augmentation and normalization for training and validation
data_transforms1 = transforms.Compose([
        # GRADED FUNCTION: Add five data augmentation methods, Normalizating and Transform to tensor
        ### START SOLUTION HERE ###
        # Add five data augmentation methods, Normalizating and Transform to tensor
        transforms.RandomRotation(45),
        transforms.CenterCrop(448),
        transforms.RandomHorizontalFlip(p=0.5),      
        transforms.RandomVerticalFlip(p=0.5),
        #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Resize(224, antialias=True),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ### END SOLUTION HERE ###
        
])
data_transforms2 = transforms.Compose([
        # GRADED FUNCTION: Add five data augmentation methods, Normalizating and Transform to tensor
        ### START SOLUTION HERE ###
        # Add five data augmentation methods, Normalizating and Transform to tensor
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ### END SOLUTION HERE ###
])

# Load Dataset and Split
Load the dataset using ImageFolder and split it into training and validation sets.

In [92]:
# Load the entire dataset
full_dataset = datasets.ImageFolder(data_dir)

# Automatically split into 80% train and 20% validation
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
train_dataset.dataset.transform = data_transforms1
val_dataset.dataset.transform = data_transforms2


# Prepare DataLoaders
Use DataLoader to prepare the training and validation datasets for model training.

In [93]:
# Use DataLoader for both train and validation datasets
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

# Get class names from the dataset
class_names = full_dataset.classes

# Load Pre-trained Model
Load a pre-trained ResNet-18 model using torchvision.models.

In [94]:
# Load pre-trained model and modify the last layer
model = models.resnet18(pretrained=True)

# Modify Model's Last Layer
Modify the last fully connected layer of the model to match the number of classes in the dataset.

In [95]:
# GRADED FUNCTION: Modify the last fully connected layer of model
### START SOLUTION HERE ###
# Modify the last fully connected layer of model

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

### END SOLUTION HERE ###

# Define Loss Function
Define the loss function to be used for training the model.

In [96]:
# GRADED FUNCTION: Define the loss function
### START SOLUTION HERE ###
# Define the loss function
criterion = nn.CrossEntropyLoss()
### END SOLUTION HERE ###

# Define Optimizer
Define the optimizer to update the model's weights during training.

In [97]:
# GRADED FUNCTION: Define the optimizer
### START SOLUTION HERE ###
# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=5e-4,momentum=0.9, nesterov=True)
### END SOLUTION HERE ###

# Define Learning Rate Scheduler
Set up a learning rate scheduler to adjust the learning rate during training.

In [98]:
# Learning rate scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Train the Model
Implement the training loop, including forward and backward passes, and save the best model based on validation accuracy.

In [99]:
# Training function
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Print learning rate for current epoch
        current_lr = optimizer.param_groups[0]['lr']
        print(f'Learning Rate: {current_lr:.6f}')

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        
                        # GRADED FUNCTION: Backward pass and optimization
                        ### START SOLUTION HERE ###
                        # Backward pass and optimization
                        
                        loss.backward()
                        optimizer.step()
                        ### END SOLUTION HERE ###

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()  # Update learning rate based on scheduler

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Save the model if validation accuracy is the best so far
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                # Save the best model
                save_dir = 'D:\\neuralnetwork\\Ex2\\work_dir'
                os.makedirs(save_dir, exist_ok=True)

               # GRADED FUNCTION: Save the best model
                ### START SOLUTION HERE ###
                # Save the best model
                torch.save(model.state_dict(), os.path.join(save_dir, 'best_model.pth'))
                ### END SOLUTION HERE ###

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    model.load_state_dict(best_model_wts)
    return model

# Train the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# 打印设备信息
if device.type == "cuda":
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")
model = model.to(device)
model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

Using GPU: NVIDIA GeForce RTX 4060 Laptop GPU


RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
